<a href="https://colab.research.google.com/github/kevinyang372/Prolog_Expert_System/blob/Sasha/LBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# !sudo apt install swi-prolog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swi-prolog is already the newest version (7.6.4+dfsg-1build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [8]:
# pip install pyswip

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Loading the data and renaming the columns 
# The names now correspond to the statement types
 
data = pd.read_csv('/content/CS152 LBA data - Sheet1.csv', skiprows=1)
col_names = ['name', 'hasCuisine', 'priceRange', 'mealTypes', 'distance',
             'isVegetarian', 'glutenFree', 'hasLanguage', 'transportation']
data.columns = col_names

In [3]:
data.head(5)

,name,hasCuisine,priceRange,mealTypes,distance,isVegetarian,glutenFree,hasLanguage,transportation
0,Miss Saigon Restaurant,Vietnamese,low,"Lunch, Dinner",17,Y,Unknown,"German, English, Vietnamese","Bus, Taxi"
1,Maroush,"Lebanese, Middle Eastern",low,"Lunch, Dinner",9,Y,Unknown,"German, English, Arabic",Walk
2,Burgermeister Kottbusser Tor,American,low,"Lunch, Dinner",12,Y,Unknown,"German, English",Walk
3,Basmah,"Sudanese, Mediterranean",low,"Lunch, Dinner",15,Y,Unknown,"German, English, Arabic","Walk, Bus"
4,Mundvoll,International,medium,"Breakfast, Brunch, Lunch, Dinner",6,Y,Y,"German, English",Walk


### Building the KB

In [0]:
# I converted Euro signs to three price levels: low, medium, high in Excel 

# all_commands = []
listCommands = []

for k in range(len(data)):
  row = data.loc[k, :]
  resName = row[0]
  # concatenate words in the name, converted to lowercase (can fix to join with lower dash later)
  resName = ''.join(e for e in resName.lower() if e.isalnum()) 
  # listCommands = [] # all commands per single restaurant name
  for i in range(1, len(row)):
    category = data.columns[i]
    items = row[i]
    if type(items) == str:
      items = row[i].split(', ')
      for item in items:
        item = ''.join(e for e in item.lower() if e.isalnum())
        command = "{0}({1}, {2})".format(category, resName, item)
        listCommands.append(command)
    else:
      command = "{0}({1}, {2})".format(category, resName, items)
      listCommands.append(command)
  # all_commands.append(listCommands) 

In [5]:
print(listCommands[12:20]) # checking to see how Middle eastern got converted

['hasCuisine(maroush, lebanese)', 'hasCuisine(maroush, middleeastern)', 'priceRange(maroush, low)', 'mealTypes(maroush, lunch)', 'mealTypes(maroush, dinner)', 'distance(maroush, 9)', 'isVegetarian(maroush, y)', 'glutenFree(maroush, unknown)']


In [0]:
from pyswip import Prolog

prolog = Prolog()

for command in listCommands:
  prolog.assertz(command)

### Queries

In [0]:
prolog.assertz("query(X, Cuisine, Price, -1) :- hasCuisine(X, Cuisine), priceRange(X, Price)")
prolog.assertz("query(X, Cuisine, Price, Distance) :- hasCuisine(X, Cuisine), priceRange(X, Price), distance(X, Y), Y < Distance")

### Test cases

In [8]:
# filter range within 10
res = set()
for soln in prolog.query("query(X, _, _, 10)"):
    res.add(soln["X"])
print(res)

{'maroush', 'mundvoll', 'lapiadina', 'santamaria', 'paglia'}


In [10]:
# filter cuisine lebanese and medium price
res = set()
for soln in prolog.query("query(X, lebanese, low, -1)"):
    res.add(soln["X"])
print(res)

{'maroush'}


In [0]:
def search(cuisine = '_', price = '_', distance = '-1'):
    to_query = "query(X, %s)" % (",".join([cuisine, price, distance]))
    return set([soln["X"] for soln in prolog.query(to_query)])

In [12]:
search()

{'amrit',
 'basmah',
 'burgermeisterkottbussertor',
 'eden',
 'hasirburger',
 'hofbraeuwirtshaus',
 'huating',
 'jambo',
 'knofifeinkost',
 'kreuzbergerweltlaterne',
 'lapiadina',
 'lemongrass',
 'longmarchcanteen',
 'mabuhay',
 'maroush',
 'milchzucker',
 'misssaigonrestaurant',
 'mundvoll',
 'nanoosh',
 'paglia',
 'pila',
 'quepasamexicana',
 'richard',
 'rissani',
 'santamaria',
 'shezan',
 'spindler',
 'sushiforyou',
 'westberlin',
 'wonderwaffel',
 'yafo'}

In [13]:
search(cuisine = "lebanese", distance = "10")

{'maroush'}